In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style="darkgrid")


from pyspark.sql.functions import *
from pyspark.sql.types import *



import folium
import html

### Load Data

##### Load business data

In [3]:
business_df = spark.read.parquet('business-small.parquet')

In [7]:
# создадим SQL вид для привычного обращения через SQL скрипт
business_df.createOrReplaceTempView("businesses")

##### Load user data

In [8]:
user_df = spark.read.parquet(data_path + 'user-small.parquet')

root
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- yelping_since: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- funny: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- fans: long (nullable = true)
 |-- average_stars: double (nullable = true)



In [11]:
user_df.createOrReplaceTempView("users")

##### Load users' friends data

In [12]:
friend_df = spark.read.parquet('friend-small.parquet')

root
 |-- user_id: string (nullable = true)
 |-- friend_id: string (nullable = true)



In [21]:
friend_df.createOrReplaceTempView("friends")

##### Load review data

In [15]:
review_df = spark.read.parquet( 'review-small.parquet')

root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: long (nullable = true)
 |-- review_date: string (nullable = true)
 |-- review_text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- funny: long (nullable = true)
 |-- cool: long (nullable = true)



In [17]:
# меняем типы данных для лучшего обращения к ним
review_df = review_df.withColumn("review_date", review_df["review_date"].cast(DateType()))

review_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: long (nullable = true)
 |-- review_date: date (nullable = true)
 |-- review_text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- funny: long (nullable = true)
 |-- cool: long (nullable = true)



In [20]:
review_df.createOrReplaceTempView("reviews")

### Простая рекомендашка (как посоветуют друзья)

In [54]:
u_id = 'uxKSnOVAoEj4I6X9YhLBlg'

query = """
select business_id, count(*) as 4_5_stars_count 
from reviews
where user_id in
    (select f.friend_id from friends f
    inner join users u on f.friend_id = u.user_id
    where f.user_id = "{}") 
and stars >= 4 
and business_id not in (select business_id from reviews where user_id = "{}")
group by business_id
order by count(*) desc limit 100
""".format(u_id, u_id)

friend_recoms_df = sqlContext.sql(query)

friend_recoms_df.toPandas()

business_id  4_5_stars_count
0   SGP1jf6k7spXkgwBlhiUVw                5
1   kOFDVcnj-8fd3doIpCQ06A                5
2   0a2O150ytxrDjDzXNfRWkA                4
3   k6zmSLmYAquCpJGKNnTgSQ                4
4   SjgeuBlgKER9yegpoxT99w                4
5   b4LmLgVdbhM-nc1IZc5Weg                3
6   -J6FVdY9pSgAdFmmalO-pQ                3
7   28adZ4lsuUeVB2aWzohK9g                3
8   G6EkDTXZ6zMUovg7JTG4YQ                3
9   RwRNR4z3kY-4OsFqigY5sw                3
10  HUYEadSbGSQNHXFmT2Ujjw                3
11  _5izyrn1qV0uyec65UsdIA                3
12  Yv4P4qUwd7F-qQ4Y4eD1JQ                3
13  _xAJZOKBMPOe47p1MphB2w                3
14  dTuT_G3Zp79RZmnF3oxfiA                3
15  GcxE5hK_TaHP4EZFDYz2mg                3
16  piZ4JqJI5WTljJuQV7yZHQ                3
17  LBHbyID7tcaGszznHtnCrw                3
18  crstB-H5rOfbXhV8pX0e6g                3
19  MhiBpIBNTCAm1Xd3WzRzjQ                3
20  TLMysu1mWiLEslCBon5W1g                2
21  RyDiwx4xD3Lx8sWHx1aFlQ                2
22  cPXasVLtq_1gmuSg65TUfQ                2
23  JJ8ypBu3b--fy4HA5RB1gg                2
24  ERnG-1q3igX3VSgm5uLZ6A                2
25  cisqOxAgQPjVVr9Yzk8_dw                2
26  mES9E5K4HbYoPFXGv0pUuw                2
27  O-uIEuv7JLUHajkemx_sVw                2
28  MC4becrrM9ntmnfMrD9b0g                2
29  SnsZFsVf1tQvBCPmhe8gGw                2
..                     ...              ...
70  Bf2fuqWbHd3L-X69FSMvmg                2
71  -Tpo5r3kF_z4iilx8JXhvA                2
72  ofw8aDSEg1HoQdmCgvLtaQ                2
73  KpFgHrxoVOiwfpR0f5KXUA                2
74  _u1Bq7QsgMUXSMex8QHqLQ                2
75  8I5U8OQ06nSxX2y4PPOWzQ                2
76  a911y0BuV1rjHXkXlFm3Ig                2
77  KvfaPsLlpOZAF276Lpvm4w                2
78  K6XIGkyk7-fuOQtA8i7p6A                2
79  0cpLF0LFQHHRFNlL0iy_Aw                2
80  RncIql4WBvbJBMed8x3F_A                2
81  4twpbw7n4DmsLxAm6-sMkg                2
82  dc3uoAmNo5STqKV6mlD_aA                2
83  9Oc8UhYNarSisYM9SEBsKQ                2
84  3tiEjjBtdfV5_npPNzs0Cg                2
85  U4iqELk1IiqARpk5uJJOZg                2
86  9_CGhHMz8698M9-PkVf0CQ                2
87  57_msr8gjNakjGO-1f6B_g                2
88  mG71gQ5HWl7ut9OCN6NJKA                2
89  DPA9MQKkCqT0qnevsG740A                2
90  oZyaFqZYf0eqKRiP-aY81Q                2
91  kDFeGBX1vWiDyJXMYRL0_A                1
92  7v91woy8IpLrqXsRvxj_vw                1
93  c1_adyjYG6JEa1PZAXMOBg                1
94  5WJOM9DefIvLOIrntDIv-A                1
95  _cjPEH9wXhKS-HQe_U3M4Q                1
96  oOGLDf2rzeCPS7UQ8hhPlQ                1
97  DIVF4oo86EGupm5CC60sZg                1
98  An_U8woQkFVsMqVblkImZQ                1
99  84ezTxVxTCmI65_UeYRJcA                1

[100 rows x 2 columns]

In [58]:
# Возьмем сэмпл данных
friend_recoms_df = friend_recoms_df.sample(False, 0.5).limit(10)
friend_recoms_df.toPandas()

business_id  4_5_stars_count
0  SGP1jf6k7spXkgwBlhiUVw                5
1  kOFDVcnj-8fd3doIpCQ06A                5
2  SjgeuBlgKER9yegpoxT99w                4
3  GcxE5hK_TaHP4EZFDYz2mg                3
4  crstB-H5rOfbXhV8pX0e6g                3
5  28adZ4lsuUeVB2aWzohK9g                3
6  HUYEadSbGSQNHXFmT2Ujjw                3
7  RwRNR4z3kY-4OsFqigY5sw                3
8  b4LmLgVdbhM-nc1IZc5Weg                3
9  a8pmtlVKf7NiSLI-4KejIw                2

In [38]:
def getBusinessDetails(in_bus):
    a = in_bus.alias("a")
    b = business_df.alias("b")
    
    return a.join(b, col("a.business_id") == col("b.business_id"), 'inner') \
             .select([col('a.'+xx) for xx in a.columns] + [col('b.business_name'),col('b.categories'),
                                                           col('b.stars'),col('b.review_count'),
                                                           col('b.latitude'),col('b.longitude')])
    

In [51]:
def showInMap(df):
    mp = folium.Map(location=[43.70011, -79.4163], zoom_start=12)

    for i, r in df.toPandas().iterrows():
        folium.Marker(
                    location =[r.latitude, r.longitude], 
                    popup = html.escape(r["business_name"]) + '<br>' + 'Stars: ' + str(r.stars) + '<br>' + 'Reviews: ' + str(r.review_count),    
                    icon = folium.Icon(color='orange')).add_to(mp)
    return mp

In [55]:
def getFriendRecoms(u_id, sim_bus_limit=10):
    
    query = """
    select business_id, count(*) as 4_5_stars_count 
    from reviews
    where user_id in
        (select f.friend_id from friends f
        inner join users u on f.friend_id = u.user_id
        where f.user_id = "{}") 
    and stars >= 4 
    and business_id not in (select business_id from reviews where user_id = "{}")
    group by business_id
    order by count(*) desc limit 100
    """.format(u_id, u_id)

    friend_recoms_df = sqlContext.sql(query)
    
    friend_recoms_df = friend_recoms_df.sample(False, 0.5).limit(sim_bus_limit)

    return getBusinessDetails(friend_recoms_df)

In [56]:
#  тестируем на одном пользователе

u_id = 'uxKSnOVAoEj4I6X9YhLBlg'

friend_recom_df = getFriendRecoms(u_id)

friend_recom_df.toPandas()

Businesses recommended to user by best useful friends:


business_id  4_5_stars_count  \
0  SGP1jf6k7spXkgwBlhiUVw                5   
1  kOFDVcnj-8fd3doIpCQ06A                5   
2  0a2O150ytxrDjDzXNfRWkA                4   
3  G6EkDTXZ6zMUovg7JTG4YQ                3   
4  RwRNR4z3kY-4OsFqigY5sw                3   
5  Yv4P4qUwd7F-qQ4Y4eD1JQ                3   
6  dTuT_G3Zp79RZmnF3oxfiA                3   
7  MhiBpIBNTCAm1Xd3WzRzjQ                3   
8  9_CGhHMz8698M9-PkVf0CQ                2   
9  ofw8aDSEg1HoQdmCgvLtaQ                2   

                              business_name  \
0                        Kekou Gelato House   
1                  Mildred's Temple Kitchen   
2                              Miku Toronto   
3                       Vietnam Noodle Star   
4         Uncle Tetsu's Japanese Cheesecake   
5                               Han Ba Tang   
6                            The Bier Markt   
7                   Messini Authentic Gyros   
8  Little Coxwell Vietnamese & Thai Cuisine   
9                        The Pie Commission   

                                          categories  stars  review_count  \
0  [Food, Restaurants, Ice Cream & Frozen Yogurt,...    4.5           332   
1  [Comfort Food, Event Planning & Services, Vege...    4.0           472   
2       [Sushi Bars, Restaurants, Seafood, Japanese]    4.0           384   
3                 [Restaurants, Vietnamese, Noodles]    3.5           148   
4  [Desserts, Japanese, Restaurants, Bakeries, Food]    3.5           806   
5  [Nightlife, Pubs, Lounges, Korean, Asian Fusio...    3.5           213   
6  [Belgian, Nightlife, Bars, Gastropubs, Canadia...    3.0           197   
7  [Mediterranean, Sandwiches, Greek, Restaurants...    3.5           372   
8                    [Vietnamese, Thai, Restaurants]    4.0           109   
9  [Canadian (New), Fast Food, Food, Do-It-Yourse...    4.5           183   

    latitude  longitude  
0  43.655983 -79.392686  
1  43.639911 -79.420424  
2  43.641235 -79.377370  
3  43.804603 -79.287842  
4  43.655969 -79.384013  
5  43.762928 -79.411511  
6  43.647095 -79.373915  
7  43.677691 -79.350536  
8  43.696175 -79.329092  
9  43.623881 -79.512074

In [57]:
showInMap(friend_recom_df)